# RQ2 Questions

In [1]:
import json
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk
import re
from array import array
from collections import defaultdict
import collections
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import math
from numpy import linalg as la
import csv
from collections import defaultdict
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Importing data

In [2]:
with open("../data/processed_tweets.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data[:30000]]

In [3]:
df_tweets = pd.DataFrame.from_records(data)

This function is to process the query

In [4]:
def processQuery(line):        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    line=  line.lower()
    line = re.sub('[^a-zA-Z]', ' ', line )                                 ## Transform in lowercase
    line=  line.split()                                 ## Tokenize the text to get a list of terms
    line=[word for word in line if word not in stops]   ##eliminate the stopwords (HINT: use List Comprehension)
    line=[stemming.stem(word) for word in line]         ## perform stemming (HINT: use List Comprehension)
    return line

I used this function to write query results to a tsv file

In [111]:
def ids2tsv(query,docs,file_name):
    with open(file_name, 'a+', encoding='utf8', newline='') as tsv_file:
        tsv_writer = csv.writer(tsv_file, delimiter='\t')
        tsv_writer.writerow([ query])
        for d_id in docs:
            text = df_tweets['text'][d_id]
            screen_name = df_tweets['user'][d_id]['screen_name']
            date = df_tweets['created_at'][d_id]
            hastags = df_tweets['hashtags'][d_id]
            likes = df_tweets['likes'][d_id]
            retweets = df_tweets['retweets'][d_id]
            followers = df_tweets["user"][d_id]['followers_count']
            user_created = df_tweets["user"][d_id]['created_at']
            tweetid =  df_tweets['id'][d_id]
            url = "https://twitter.com/{}/status/{}".format(screen_name,tweetid)
            tsv_writer.writerow(("", text, screen_name, date,hastags, likes, retweets, url, tweet2label[d_id] ))
        tsv_writer.writerow(["", ""])
    

## Clustering

For clustering, I used word2vec representation as in question 1

In [45]:
all_words = []
for terms in df_tweets['processed_text']:
    for word in terms:
        all_words.append(nltk.word_tokenize(word))

In [63]:
word2vec = Word2Vec(all_words, size=24, min_count=1)

Generate tweets vector based on word2vec representation

In [64]:
tweet2vec = {}
tweetvecList = []
key = 0
for terms in df_tweets['processed_text']:
    tweet2vec[key] = 0
    for word in terms:
        try:
            tweet2vec[key] += (word2vec[word])
        except:
            continue
    if(len(terms) > 0):
        tweet2vec[key] = tweet2vec[key] / len(terms)
        tweet2vec[key] = (tweet2vec[key])
        tweetvecList.append(tweet2vec[key])
    key+=1

Generating clusters

In [65]:
from sklearn.cluster import KMeans
from collections import Counter

In [66]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(tweetvecList)

In [67]:
labels = kmeans.labels_

In [69]:
labels2tweets=defaultdict(lambda: [])

In [70]:
for pos, label in enumerate(labels):
    labels2tweets[label].append(pos)

In [71]:
def bag_of_words_from_ids(ids):
    bag_of_words = {}
    for key in ids:
        lst_text = df_tweets['processed_text'][key]
        # create bag-of-words - for each word the frequency of the word in the corpus
        for w in lst_text:
            if w not in bag_of_words:
                bag_of_words[w] = 0
            bag_of_words[w]+=1

    return bag_of_words

In [72]:
def get_head(bag):
    words_obj = [{"word":word, "count": bag[word]} for word in bag]

    counter_words = pd.DataFrame(words_obj)

    counter_words.sort_values("count", inplace=True, ascending=False)

    total_count = 0
    counter_words = counter_words.head(20)
    for index, row in counter_words.iterrows():
        total_count += row['count']
    word2norm = []
    for index, row in counter_words.iterrows():
        word2norm.append(row['count'] / total_count)
    counter_words['norm'] = word2norm
    return counter_words

Generating cluster label to terms head

In [73]:
label2head = {}
for l in labels2tweets:
    bag_of_words =  bag_of_words_from_ids(labels2tweets[l])
    label2head[l] = get_head(bag_of_words)

Dictionary for storing get the cluster label from tweet

In [74]:
tweet2label = {}
for index, row in df_tweets.iterrows():
    tweet2label[index] = -1
    for label in labels2tweets:
        if(index in labels2tweets[label]):
            tweet2label[index] = label
            break

## Diversity Score

Diversity score function

In [80]:
def compute_diversity_score():
    ds = defaultdict(list)
    #iterate rows
    for index, row in df_tweets.iterrows():
        score = []
        label = tweet2label[index]
        if(label != -1): 
            score.append(label)
            head = label2head[label]
            ds_terms = {}
            #iterate terms in eact tweet and compute term frequency fro terms in the cluster head
            for term in row['processed_text']:
                if(term in list(head['word'])):
                    if(term in ds_terms):
                        ds_terms[term] += 1
                    else:
                        ds_terms[term] = 1
            #iterate the terms from before and normalize them by multiplying with the normalization compute for each term
            for i, row in head.iterrows():
                if(row['word'] in ds_terms):
                    ds_terms[row['word']] = ds_terms[row['word']] * row['norm']
            tweetds = 0
            #sum all the scores and divide by number of head terms' in the tweet
            for term in ds_terms:
                tweetds += ds_terms[term]
            score.append(tweetds / len(ds_terms))
        else:
            #if label no from any cluster
            score = [-1,0]
        ds[index] = score
    return ds


In [81]:
ds = compute_diversity_score()

In [88]:
def create_index_tfidf(df_tweets):
    numDocuments = len(df_tweets)
    index=defaultdict(list)

    tf=defaultdict(list) 
    df=defaultdict(int)        
    idf=defaultdict(float)


    for i, row in df_tweets.iterrows():
        terms = row['processed_text']

        page_id =  i
        termdictPage={}

        for position, term in enumerate(terms):
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] 
        
        norm=0
        for term, posting in termdictPage.items(): 
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)

        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():   
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] = df[term] + 1  

        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
    for term in df:
        idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
            
    return index, tf, df, idf


In [89]:
index, tf, df, idf = create_index_tfidf(df_tweets)

In [122]:
def get_label_from_query(query):
    labels = []
    for term in query:
        for label in label2head:
            for index, row in label2head[label].iterrows():
                if(term in row['word'] and label not in labels):
                    labels.append(label)
                        
    return labels

In [ ]:
def rankDocuments(terms, docs, index, idf, tf):
    docVectors=defaultdict(lambda: [0]*len(terms)) 

    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) 
    
    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms): 
        if term not in index:
            continue
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term] 

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
               
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term] 
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]

    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]

    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    
    return resultDocs

In [101]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=processQuery(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs|=set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf)   
    return ranked_docs

Funciton to rerank based on the diversity score

In [123]:
def diversification(rankedDocs):
    docDivScores = [[ds[d_id][1], d_id] for d_id in rankedDocs]
    print(docDivScores)
    docDivScores.sort(reverse=True)
    resultDocs=[x[1] for x in docDivScores]
    return resultDocs

## Write tsv files

In [112]:
queries=[
    "us lockdown",
    "trump has covid",
    "holidays with covid",
    "the pandemic in us"
]

In [116]:
file_nameRQ2A_no_ds= "../other-outputs/RQ2A_no_ds.tsv"
file_nameRQ2A_ds="../other-outputs/RQ2A_ds.tsv"

In [117]:
with open(file_nameRQ2A_no_ds, 'a+', encoding='utf8', newline='') as tsv_file:
    tsv_writer = csv.writer(tsv_file, delimiter='\t')
    tsv_writer.writerow(["Query","Tweet", "Username", "Date", "Hashtags","Likes","Retweets","Url","Cluster"])

In [119]:
for query in queries:
    print(query)
    ranked_docs = search_tf_idf(query, index)
    ids2tsv(query, ranked_docs[:20], file_nameRQ2A_no_ds)

us lockdown
trump has covid
holidays with covid
the pandemic in us


In [121]:
with open(file_nameRQ2A_ds, 'a+', encoding='utf8', newline='') as tsv_file:
    tsv_writer = csv.writer(tsv_file, delimiter='\t')
    tsv_writer.writerow(["Query","Tweet", "Username", "Date", "Hashtags","Likes","Retweets","Url","Cluster"])

In [122]:
for query in queries:
    print(query)
    ranked_docs = search_tf_idf(query, index)
    ranked_docs = diversification(ranked_docs)
    ids2tsv(query, ranked_docs[:20], file_nameRQ2A_ds)

us lockdown
trump has covid
holidays with covid
the pandemic in us
